<h2>Read Data

In [1]:
import pandas as pd
import numpy as np
import talib

In [2]:
PVH = pd.read_csv('PVH.csv',index_col=0)
NKE = pd.read_csv('NKE.csv',index_col=0) 
GPC = pd.read_csv('GPC.csv',index_col=0)
TSCO = pd.read_csv('TSCO.csv',index_col=0)
WMT = pd.read_csv('WMT.csv',index_col=0)
SHW = pd.read_csv('SHW.csv',index_col=0)
HLT = pd.read_csv('HLT.csv',index_col=0)
MAR = pd.read_csv('MAR.csv',index_col=0)
CCL = pd.read_csv('CCL.csv',index_col=0)
NCLH = pd.read_csv('NCLH.csv',index_col=0)
RCL = pd.read_csv('RCL.csv',index_col=0)
V = pd.read_csv('V.csv',index_col=0)
MA = pd.read_csv('MA.csv',index_col=0)

In [3]:

CPRI = pd.read_excel('cpri_bb.xlsx',index_col=0)
CPRI = CPRI.rename(columns={'PX_OPEN':'Open','PX_HIGH':'High',
                           'PX_LOW':'Low','PX_LAST':'Close','PX_VOLUME':'Volume'})
CPRI

,Open,High,Low,Close,Volume
Dates,,,,,
2011-12-15,25.00,25.23,23.51,24.20,42261341
2011-12-16,24.45,24.80,23.51,24.10,3998809
2011-12-19,24.50,25.09,24.31,24.88,3246130
2011-12-20,25.02,25.76,25.02,25.45,1710133
2011-12-21,25.64,26.78,25.46,26.55,2501413
...,...,...,...,...,...
2019-11-11,36.35,36.50,35.93,36.36,2552326
2019-11-12,36.65,38.06,36.20,36.55,2587964
2019-11-13,36.20,36.20,35.51,35.93,1820703


<h2>MACD: Moving Average Convergence Divergence</h2>

The default parameters used are (12,26,9):
\begin{align}
&MACD = EMA_{12}(CLOSE) - EMA_{26}(CLOSE)\\
&Signal = EMA_{9}(MACD)
\end{align}

<b>Usage</b>: 
<ol>
<li>Buy signal: when MACD line crosses above the signal line;
<li>Sell/short signal: when MACK line crosses below the signal line;
</ol>
Actually, it is the change of macdhist that matters.

<b>Other parameters</b>: (8,17,9),(3,10,16) and (5,35,5); 

In [5]:
parameters = [(12,26,9),(24,52,9),(5,42,5),(34,89,34),(5,26,5)]

In [6]:
# For macd, the position of the first non-NaN value is the same as position for the first non-NaN valoe of macdsignal

def MACD(stock,parameters):
    """
    1. Calculate MACD/MACDSignal/MACDHist with given parameters;
    2. Merge the Calculated MACDs to the originial dataframe;
    """
    
    for parameter in parameters:
        fastperiod = parameter[0]
        slowperiod = parameter[1]
        signalperiod = parameter[2]
        
        macd, macdsignal, macdhist = talib.MACD(stock['Close'], fastperiod=fastperiod,
                                                slowperiod=slowperiod, signalperiod=signalperiod)
        stock[f'MACD{fastperiod,slowperiod,signalperiod}'] = macd
        stock[f'MACDSignal{fastperiod,slowperiod,signalperiod}'] = macdsignal
    
    return stock

In [7]:
PVH = MACD(PVH,parameters)
NKE = MACD(NKE,parameters) 
GPC = MACD(GPC,parameters)
TSCO = MACD(TSCO,parameters)
WMT = MACD(WMT,parameters)
SHW = MACD(SHW,parameters)
HLT = MACD(HLT,parameters)
MAR = MACD(MAR,parameters)
CCL = MACD(CCL,parameters)
NCLH = MACD(NCLH,parameters)
RCL = MACD(RCL,parameters)
V = MACD(V,parameters)
MA = MACD(MA,parameters)
CPRI = MACD(CPRI,parameters)
PVH_new = MACD(PVH_new,parameters)

<h2> RSI: Relative Strength Index</h2>

The default parameter for RSI is 14.

\begin{align}
    &RS = \frac{average\ gain\ over\ last\ 14\ days}{average\ loss\ over\ the\ last\ 14\ days} \\
    &RSI = 100 - 100/(1+RS)
\end{align}

<b>Usage:</b> detect overbought/oversold;

Overbought: >70;
Oversold: <30;

This indicator works better in the <b>choppy market</b>.

<b>Other parameters</b>: 6/8/10/20

Because we don't have return value on the initial date, the first value of RSI appears on the n+1 date.

In [8]:
parameters = [5,10,14,20,26]

In [9]:
def RSI(stock,parameters):
    """
    1. Calculate RSI with given parameters;
    2. Merge the Calculated RSI to the originial dataframe;
    """ 
    
    for parameter in parameters:
        rsi = talib.RSI(stock['Close'],timeperiod=parameter)
        stock[f'RSI{parameter}'] = rsi
        
    return stock

In [10]:
PVH = RSI(PVH,parameters)
NKE = RSI(NKE,parameters) 
GPC = RSI(GPC,parameters)
TSCO = RSI(TSCO,parameters)
WMT = RSI(WMT,parameters)
SHW = RSI(SHW,parameters)
HLT = RSI(HLT,parameters)
MAR = RSI(MAR,parameters)
CCL = RSI(CCL,parameters)
NCLH = RSI(NCLH,parameters)
RCL = RSI(RCL,parameters)
V = RSI(V,parameters)
MA = RSI(MA,parameters)
CPRI = RSI(CPRI,parameters)
PVH_new = RSI(PVH_new,parameters)

<h2>ROC: price rate of change</h2>

reflects the percentage change in closing price.
\begin{equation}
ROC_n(t) = \frac{Close(t)-Close(t-n)}{Close(t-n)}*100
\end{equation}

<b>Usage</b>: 
<ol>
    <li>zeroline crossover generates buy/sell signal.</li>
    <li>determineoverbought/oversold;</li>
</ol>

<b>Other parameters: 6/10/12/20/30

In [11]:
parameters = [5,10,14,20,26]

In [12]:
def ROC(stock,parameters):
    """
    1. Calculate ROC with given parameters;
    2. Merge the Calculated ROC to the originial dataframe;
    """ 
    for parameter in parameters:
        roc = talib.ROC(stock['Close'],timeperiod=parameter)
        stock[f'ROC{parameter}'] = roc
    return stock

In [13]:
PVH = ROC(PVH,parameters)
NKE = ROC(NKE,parameters) 
GPC = ROC(GPC,parameters)
TSCO = ROC(TSCO,parameters)
WMT = ROC(WMT,parameters)
SHW = ROC(SHW,parameters)
HLT = ROC(HLT,parameters)
MAR = ROC(MAR,parameters)
CCL = ROC(CCL,parameters)
NCLH = ROC(NCLH,parameters)
RCL = ROC(RCL,parameters)
V = ROC(V,parameters)
MA = ROC(MA,parameters)
CPRI = ROC(CPRI,parameters)
PVH_new = ROC(PVH_new,parameters)

<h2>Stochastic Oscillator</h2>

measures the level of the closing price relative to low-high range over a period of time.

Default parameter: (14,3).

\begin{align}
&\%K=\frac{Close-Low_{14}}{High_{14}-Low_{14}} \\
&\%D = 3\ days\ moving\ average\ of\ \%K
\end{align}

This indicator is predicted on the assumption that close prices should close near the same direction as the current trend.

<b>Usage:</b>
<ol>
    <li>%K greater than 80:overbought;%K less than 20:oversold;
    <li>Buy signal: %K crosses up %D; Sell/short signal: %K crosses down %D;
</ol>

Therefore this indicator works better in <b>trending market</b>.


In [14]:
parameters = [(5,3),(10,5),(14,5),(21,7),(21,14)]

In [15]:
def STOCH(stock,parameters):
    """
    1. Calculate STOCH with given parameters;
    2. Merge the Calculated STOCH to the originial dataframe;
    """ 
    for parameter in parameters:
        fastk_period = parameter[0]
        fastd_period = parameter[1]
        fastk, fastd = talib.STOCHF(stock['High'], stock['Low'], stock['Close'], 
                              fastk_period=fastk_period, fastd_period=fastd_period, fastd_matype=0)
        stock[f'%K{fastk_period,fastd_period}'] = fastk
        stock[f'%D{fastk_period,fastd_period}'] = fastd
    return stock

In [16]:
PVH = STOCH(PVH,parameters)
NKE = STOCH(NKE,parameters) 
GPC = STOCH(GPC,parameters)
TSCO = STOCH(TSCO,parameters)
WMT = STOCH(WMT,parameters)
SHW = STOCH(SHW,parameters)
HLT = STOCH(HLT,parameters)
MAR = STOCH(MAR,parameters)
CCL = STOCH(CCL,parameters)
NCLH = STOCH(NCLH,parameters)
RCL = STOCH(RCL,parameters)
V = STOCH(V,parameters)
MA = STOCH(MA,parameters)
CPRI = STOCH(CPRI,parameters)
PVH_new = STOCH(PVH_new,parameters)

<h2>Williams' %R</h2>

\begin{equation}
\%R = -100*\frac{High_{14}-Close}{High_{14}-Low_{14}}
\end{equation}

Because it is similar to stochastic oscillator, we only set parameter to 14.

In [17]:
parameters = [5,10,14,20,28]

In [18]:
def WILLR(stock,parameters):
    """
    1. Calculate WILLR with given parameters;
    2. Merge the Calculated WILLR to the originial dataframe;
    """ 
    for parameter in parameters:
        WilliamsR = talib.WILLR(stock['High'], stock['Low'], stock['Close'], timeperiod=parameter)
        stock[f'WilliamsR{parameter}'] = WilliamsR
    return stock

In [19]:
PVH = WILLR(PVH,parameters)
NKE = WILLR(NKE,parameters) 
GPC = WILLR(GPC,parameters)
TSCO = WILLR(TSCO,parameters)
WMT = WILLR(WMT,parameters)
SHW = WILLR(SHW,parameters)
HLT = WILLR(HLT,parameters)
MAR = WILLR(MAR,parameters)
CCL = WILLR(CCL,parameters)
NCLH = WILLR(NCLH,parameters)
RCL = WILLR(RCL,parameters)
V = WILLR(V,parameters)
MA = WILLR(MA,parameters)
CPRI = WILLR(CPRI,parameters)
PVH_new = WILLR(PVH_new,parameters)

<h2>OBV: On Balance Volume</h2>

\begin{align}
OBV_t = OBV_{t-1}+VOL_t (Close_t > Close_{t-1})  \\
OBV_t = OBV_{t-1}-VOL_t (Close_t < Close_{t-1}) 
\end{align}

Look at the Volume

In [20]:
def OBV(stock,parameters):
    """
    1. Calculate OBV with given parameters;
    2. Merge the Calculated OBV to the originial dataframe;
    """ 
    obv = talib.OBV(stock['Close'], stock['Volume'])
    stock['OBV'] = obv
    return stock

In [21]:
PVH = OBV(PVH,parameters)
NKE = OBV(NKE,parameters) 
GPC = OBV(GPC,parameters)
TSCO = OBV(TSCO,parameters)
WMT = OBV(WMT,parameters)
SHW = OBV(SHW,parameters)
HLT = OBV(HLT,parameters)
MAR = OBV(MAR,parameters)
CCL = OBV(CCL,parameters)
NCLH = OBV(NCLH,parameters)
RCL = OBV(RCL,parameters)
V = OBV(V,parameters)
MA = OBV(MA,parameters)
CPRI = OBV(CPRI,parameters)
PVH_new = OBV(PVH_new,parameters)

<h2>Bollinger Bands</h2>

It gives the moving average, an upper bound and a lower bound.(2 sigma wide for each side)

Middle: 20-days simple moving average

<b>Usage:</b>
<ol>
<li>The closer the price moves to the upper bound, the more overbought the market;
<li>The closer the price moves to the lower bound, the more oversold the market.
</ol>

In [22]:
parameters = [10,15,20,25,30]

In [23]:
def BB(stock,parameters):
    """
    1. Calculate Bollinger Bands with given parameters;
    2. Merge the Calculated Bollinger Bands to the originial dataframe;
    """
    for parameter in parameters:
        upper,middle,lower = talib.BBANDS(stock['Close'], timeperiod=parameter, nbdevup=2, nbdevdn=2, matype=0)
        stock[f'UpperBound{parameter}'] = upper
        stock[f'Middle{parameter}'] = middle
        stock[f'LowerBound{parameter}'] = lower
    return stock

In [24]:
PVH = BB(PVH,parameters)
NKE = BB(NKE,parameters) 
GPC = BB(GPC,parameters)
TSCO = BB(TSCO,parameters)
WMT = BB(WMT,parameters)
SHW = BB(SHW,parameters)
HLT = BB(HLT,parameters)
MAR = BB(MAR,parameters)
CCL = BB(CCL,parameters)
NCLH = BB(NCLH,parameters)
RCL = BB(RCL,parameters)
V = BB(V,parameters)
MA = BB(MA,parameters)
CPRI = BB(CPRI,parameters)
PVH_new = BB(PVH_new,parameters)

<h2>CCI:Commodity Channel Index)</h2>
    
measures the current price level relative to an average price of a given period.

\begin{align}
&CCI = \frac{TP-MA}{0.015*Mean\ Deviation} \\
&TP = \frac{Low+High+Close}{3} \\
&MA = 20-days\ SMA\ of\ TP \\
&Mean\ Deviation = \sum_{n}|TP-MA|/n
\end{align}


Default parameter: 20

<b>Usage:</b>
Overbought: CCI<-200 or -100; oversold: CCI>200 or 100



In [25]:
parameters = [10,15,20,30,40]

In [26]:
def CCI(stock,parameters):
    """
    1. Calculate CCI with given parameters;
    2. Merge the Calculated CCI to the originial dataframe;
    """
    for parameter in parameters:
        cci = talib.CCI(stock['High'], stock['Low'], stock['Close'], timeperiod=parameter)
        stock[f'CCI{parameter}'] = cci
    return stock

In [27]:
PVH = CCI(PVH,parameters)
NKE = CCI(NKE,parameters) 
GPC = CCI(GPC,parameters)
TSCO = CCI(TSCO,parameters)
WMT = CCI(WMT,parameters)
SHW = CCI(SHW,parameters)
HLT = CCI(HLT,parameters)
MAR = CCI(MAR,parameters)
CCL = CCI(CCL,parameters)
NCLH = CCI(NCLH,parameters)
RCL = CCI(RCL,parameters)
V = CCI(V,parameters)
MA = CCI(MA,parameters)
CPRI = CCI(CPRI,parameters)
PVH_new = CCI(PVH_new,parameters)

<h2>ATR: Average True Range</h2>

measures the market volatility by decomposing the entire range of an asset price for that period.

\begin{align}
&TR = max(high,close_{prev})-min(low,close_{prev}) \\
&ATR_t = \frac{ATR_{t-1}*(n-1)+TR_t}{n} \\
\end{align}



In [28]:
parameters = [5,10,14,21,63]

In [29]:
def ATR(stock,parameters):
    """
    1. Calculate ATR with given parameters;
    2. Merge the Calculated ATR to the originial dataframe;
    """
    for parameter in parameters:
        atr = talib.ATR(stock['High'], stock['Low'], stock['Close'], timeperiod=parameter)
        stock[f'ATR{parameter}'] = atr
    return stock

In [30]:
PVH = ATR(PVH,parameters)
NKE = ATR(NKE,parameters) 
GPC = ATR(GPC,parameters)
TSCO = ATR(TSCO,parameters)
WMT = ATR(WMT,parameters)
SHW = ATR(SHW,parameters)
HLT = ATR(HLT,parameters)
MAR = ATR(MAR,parameters)
CCL = ATR(CCL,parameters)
NCLH = ATR(NCLH,parameters)
RCL = ATR(RCL,parameters)
V = ATR(V,parameters)
MA = ATR(MA,parameters)
CPRI = ATR(CPRI,parameters)
PVH_new = ATR(PVH_new,parameters)

<h2>ADX: Average Directional Momentum Index</h2>

<b>Usage:</b> identify whether it is a trending market.

It can be used with Plus directional indicator and Minus direcational indicator to generate buy/sell signal.

In [31]:
parameters = [5,10,14,21,63]

In [32]:
def ADX(stock,parameters):
    """
    1. Calculate ADX with given parameters;
    2. Merge the Calculated ADX to the originial dataframe;
    """
    for parameter in parameters:
        adx = talib.ADX(stock['High'], stock['Low'], stock['Close'], timeperiod=parameter)
        pdi = talib.PLUS_DI(stock['High'], stock['Low'], stock['Close'], timeperiod=parameter)
        mdi = talib.MINUS_DI(stock['High'], stock['Low'], stock['Close'], timeperiod=parameter)
        stock[f'ADX{parameter}'] = adx
        stock[f'PDI{parameter}'] = pdi
        stock[f'MDI{parameter}'] = mdi
    return stock

In [33]:
PVH = ADX(PVH,parameters)
NKE = ADX(NKE,parameters) 
GPC = ADX(GPC,parameters)
TSCO = ADX(TSCO,parameters)
WMT = ADX(WMT,parameters)
SHW = ADX(SHW,parameters)
HLT = ADX(HLT,parameters)
MAR = ADX(MAR,parameters)
CCL = ADX(CCL,parameters)
NCLH = ADX(NCLH,parameters)
RCL = ADX(RCL,parameters)
V = ADX(V,parameters)
MA = ADX(MA,parameters)
CPRI = ADX(CPRI,parameters)
PVH_new = ADX(PVH_new,parameters)

<h2>MFI: money flow index</h2>

is used to identify the overbought/oversold condition.

Default parameter: 14

\begin{align}
&TP = \frac{High+Low+Close}{3} \\
&Raw\ Money\ Flow = TP*Volume \\
&Money\ Flow\ Ratio = \frac{14\ period\ positive\ money\ flow}{14\ period\ negative\ money\ flow}
\end{align}


In [34]:
parameters = [5,10,14,21,63]

In [35]:
def MFI(stock,parameters):
    """
    1. Calculate MFI with given parameters;
    2. Merge the Calculated MFI to the originial dataframe;
    """
    for parameter in parameters:
        mfi = talib.MFI(stock['High'], stock['Low'], stock['Close'], stock['Volume'], timeperiod=parameter)
        stock[f'MFI{parameter}'] = mfi
    return stock

In [36]:
PVH = MFI(PVH,parameters)
NKE = MFI(NKE,parameters) 
GPC = MFI(GPC,parameters)
TSCO = MFI(TSCO,parameters)
WMT = MFI(WMT,parameters)
SHW = MFI(SHW,parameters)
HLT = MFI(HLT,parameters)
MAR = MFI(MAR,parameters)
CCL = MFI(CCL,parameters)
NCLH = MFI(NCLH,parameters)
RCL = MFI(RCL,parameters)
V = MFI(V,parameters)
MA = MFI(MA,parameters)
CPRI = MFI(CPRI,parameters)
PVH_new = MFI(PVH_new,parameters)

<h2>Momentum</h2>

\begin{equation}
mtm = Close_t-Close_{t-n}
\end{equation}
just the price change in last n period.

In [37]:
parameters = [5,10,20,50,100,200]

In [38]:
def MTM(stock,parameters):
    """
    1. Calculate MTM with given parameters;
    2. Merge the Calculated MTM to the originial dataframe;
    """
    for parameter in parameters:
        mtm = talib.MOM(stock['Close'], timeperiod=parameter)
        stock[f'MTM{parameter}'] = mtm
    return stock

In [39]:
PVH = MTM(PVH,parameters)
NKE = MTM(NKE,parameters) 
GPC = MTM(GPC,parameters)
TSCO = MTM(TSCO,parameters)
WMT = MTM(WMT,parameters)
SHW = MTM(SHW,parameters)
HLT = MTM(HLT,parameters)
MAR = MTM(MAR,parameters)
CCL = MTM(CCL,parameters)
NCLH = MTM(NCLH,parameters)
RCL = MTM(RCL,parameters)
V = MTM(V,parameters)
MA = MTM(MA,parameters)
CPRI = MTM(CPRI,parameters)
PVH_new = MTM(PVH_new,parameters)

<h2>CMO: Chande Momentum Oscillator</h2>

\begin{equation}
CMO = 100*\frac{S_u-S_d}{S_u+S_d}
\end{equation}

$S_u$: Sum of the difference between the current close and previous close on up days for the specified period.

$S_d$:  Sum of the absolute value of the difference between the current close and the previous close on down days for the specified period.

<b>Usage:</b>
<ol>
    <li>Identify overbought/oversold condition
    <li>Measures the trend strength
</ol>

In [40]:
parameters = [5,10,14,21,63]

In [41]:
def CMO(stock,parameters):
    """
    1. Calculate CMO with given parameters;
    2. Merge the Calculated CMO to the originial dataframe;
    """
    for parameter in parameters:
        cmo = talib.CMO(stock['Close'], timeperiod=parameter)
        stock[f'CMO{parameter}'] = cmo
    return stock

In [42]:
PVH = CMO(PVH,parameters)
NKE = CMO(NKE,parameters) 
GPC = CMO(GPC,parameters)
TSCO = CMO(TSCO,parameters)
WMT = CMO(WMT,parameters)
SHW = CMO(SHW,parameters)
HLT = CMO(HLT,parameters)
MAR = CMO(MAR,parameters)
CCL = CMO(CCL,parameters)
NCLH = CMO(NCLH,parameters)
RCL = CMO(RCL,parameters)
V = CMO(V,parameters)
MA = CMO(MA,parameters)
CPRI = CMO(CPRI,parameters)
PVH_new = CMO(PVH_new,parameters)

<h2>TRIX: triple exponential average</h2>

\begin{align}
M = EMA(EMA(EMA(Close))) \\
TRIX = \frac{M_t-M_{t-1}}{M_{t-1}}
\end{align}

Identify overbought/oversold condition.

Compare with the signal line(9-days EMA of TRIX) to generate buy/sell signal

Many analysts believe that when the TRIX crosses above the zero line it gives a buy signal, and when it closes below the zero line, it gives a sell signal.

In [43]:
parameters = [5,10,14,20,28]

In [44]:
def TRIX(stock,parameters):
    """
    1. Calculate TRIX with given parameters;
    2. Merge the Calculated TRIX to the originial dataframe;
    """
    for parameter in parameters:
        trix = talib.TRIX(stock['Close'], timeperiod=parameter)
        stock[f'TRIX{parameter}'] = trix
    return stock

In [45]:
PVH = TRIX(PVH,parameters)
NKE = TRIX(NKE,parameters) 
GPC = TRIX(GPC,parameters)
TSCO = TRIX(TSCO,parameters)
WMT = TRIX(WMT,parameters)
SHW = TRIX(SHW,parameters)
HLT = TRIX(HLT,parameters)
MAR = TRIX(MAR,parameters)
CCL = TRIX(CCL,parameters)
NCLH = TRIX(NCLH,parameters)
RCL = TRIX(RCL,parameters)
V = TRIX(V,parameters)
MA = TRIX(MA,parameters)
CPRI = TRIX(CPRI,parameters)
PVH_new = TRIX(PVH_new,parameters)

<h2>AD:Chaikin Accumulation Distribution Line Oscillator</h2>

measure the cumulative flow of money into and out of a security.

\begin{align}
&CLV=\frac{((close-low)-(high-close))}{(high-low)} \\
&AD={AD_{-1}}+CLV*volume
\end{align}

<b>usage:</b>
1. identify the price trend;
2. If the price is rising but A/D is falling, it signals underlying weakness and a potential decline in price;
   If the price of an asset is falling but A/D is rising, it signals underlying strength and the price may start to rise.

In [46]:
parameters = [(3,10),(6,20)]

In [47]:
def AD(stock,parameters):
    """
    1. Calculate AD with given parameters;
    2. Merge the Calculated AD to the originial dataframe;
    """
    for parameter in parameters:
        ad = talib.ADOSC(stock['High'], stock['Low'], stock['Close'], stock['Volume'], 
                         fastperiod=parameter[0], slowperiod=parameter[1])
        stock[f'AD{parameter[0],parameter[1]}'] = ad
    return stock

In [48]:
PVH = AD(PVH,parameters)
NKE = AD(NKE,parameters) 
GPC = AD(GPC,parameters)
TSCO = AD(TSCO,parameters)
WMT = AD(WMT,parameters)
SHW = AD(SHW,parameters)
HLT = AD(HLT,parameters)
MAR = AD(MAR,parameters)
CCL = AD(CCL,parameters)
NCLH = AD(NCLH,parameters)
RCL = AD(RCL,parameters)
V = AD(V,parameters)
MA = AD(MA,parameters)
CPRI = AD(CPRI,parameters)
PVH_new = AD(PVH_new,parameters)

<h2>VHF:Vertical Horizontal Filter </h2>

determines whether prices are in a trending phase or a trading range or congestion phase

\begin{align}
&HCP: highest \ closing \ price \\
&LCP: lowest \ closing \ price \\
&VHF =\frac{ (HCP - LCP)} {Sum \ of \ absolute \ values \ for \ n \ periods}
\end{align}

usage:
1. determines whether signals are taken from trend or momentum indicators;
2. Rising values indicate a trend; Falling values indicate a ranging market; High values precede the end of a trend; Low values precede a trend start.

In [49]:
parameters=[5,10,14,20,28]

In [50]:
def VHF(stock,parameters):
    """
    1. Calculate vhf with given parameters;
    2. Merge the Calculated VHF to the originial dataframe; 
    """
    for parameter in parameters:
        LCP = talib.MIN(stock['Close'], timeperiod=parameter)
        HCP = talib.MAX(stock['Close'], timeperiod=parameter)
        NUM = HCP - LCP
        DEN = abs(stock['Close']-stock['Close'].shift())
        DEN = talib.MA(DEN, timeperiod=parameter)*parameter
        stock[f'VHF{parameter}']=NUM.div(DEN)
    return stock

In [51]:
PVH = VHF(PVH,parameters)
NKE = VHF(NKE,parameters) 
GPC = VHF(GPC,parameters)
TSCO = VHF(TSCO,parameters)
WMT = VHF(WMT,parameters)
SHW = VHF(SHW,parameters)
HLT = VHF(HLT,parameters)
MAR = VHF(MAR,parameters)
CCL = VHF(CCL,parameters)
NCLH = VHF(NCLH,parameters)
RCL = VHF(RCL,parameters)
V = VHF(V,parameters)
MA = VHF(MA,parameters)
CPRI = VHF(CPRI,parameters)
PVH_new = VHF(PVH_new,parameters)

<h2>BIAS:Bias Ratio</h2>

measures how far the returns from an investment portfolio

\begin{align}
&BIAS=\frac{(Close-MA)}{MA}* 100% 
\end{align}

<b>usage:</b>
1. reflects the possibility of price backlash or rebound due to deviation from the moving average trend when the price fluctuates sharply;
2. reflects the credibility that the price moves within the normal fluctuation range to continue the original trend;

In [52]:
parameters=[5,10,14,20,28]

In [53]:
def BIAS(stock,parameters):
    """
    1. Calculate bias ratio with given parameters;
    2. Merge the Calculated bias ratio to the originial dataframe; 
    """
    for parameter in parameters:
        MA = talib.MA(stock['Close'],timeperiod=parameter)
        stock[f'BIAS{parameter}']=(stock['Close']-MA)/MA
    return stock

In [54]:
PVH = BIAS(PVH,parameters)
NKE = BIAS(NKE,parameters) 
GPC = BIAS(GPC,parameters)
TSCO = BIAS(TSCO,parameters)
WMT = BIAS(WMT,parameters)
SHW = BIAS(SHW,parameters)
HLT = BIAS(HLT,parameters)
MAR = BIAS(MAR,parameters)
CCL = BIAS(CCL,parameters)
NCLH = BIAS(NCLH,parameters)
RCL = BIAS(RCL,parameters)
V = BIAS(V,parameters)
MA = BIAS(MA,parameters)
CPRI = BIAS(CPRI,parameters)
PVH_new = BIAS(PVH_new,parameters)

<h2>DPO:detrended price oscillator</h2>

measures the distance between peaks and troughs in the price/indicator.

\begin{align}
Close(t-(1+\frac{n}{2})) - SMA(n).
\end{align}

<b>usage:</b> estimates future peaks to sell or the estimated future troughs to buy



In [55]:
parameters=[5,10,14,20,28]

In [56]:
def DPO(stock,parameters):
    """
    1. Calculate DPO with given parameters;
    2. Merge the Calculated DPO to the originial dataframe; 
    """
    for parameter in parameters:
        p = talib.MA(stock['Close'], timeperiod=parameter)
        shift_days = int(1 + parameter/2)
        close_shifted = stock['Close'].shift(periods=shift_days)
        stock[f'DPO{parameter}']=close_shifted-p
    return stock

In [57]:
PVH = DPO(PVH,parameters)
NKE = DPO(NKE,parameters) 
GPC = DPO(GPC,parameters)
TSCO = DPO(TSCO,parameters)
WMT = DPO(WMT,parameters)
SHW = DPO(SHW,parameters)
HLT = DPO(HLT,parameters)
MAR = DPO(MAR,parameters)
CCL = DPO(CCL,parameters)
NCLH = DPO(NCLH,parameters)
RCL = DPO(RCL,parameters)
V = DPO(V,parameters)
MA = DPO(MA,parameters)
CPRI = DPO(CPRI,parameters)
PVH_new = DPO(PVH_new,parameters)

<h2>PSY:Psychological Line</h2>

reflects the buying power in relation to the selling power

PSY the percentage of bars that close above the previous candle close within the PLI calculation period. 

<b>usage</b>:
1. relfects who is in control:If PSY is above 50%, it indicates that buyers are in control. Likewise, if it is below 50%, it indicates the sellers are in control. 
2. reflects divergence signal:When the stock prices make a new high (low) while PSY does not make a new high (low), it signals the divergence.

In [58]:
parameters = [5,10,14,20,28]

In [59]:
def PSY(stock,parameters):
    """
    1. Calculate PSY with given parameters;
    2. Merge the Calculated PSY to the originial dataframe; 
    """
    for parameter in parameters:
        priceData=stock['Close'].values
        difference = priceData[1:] - priceData[:-1]
        difference = np.append(0, difference)
        difference_dir = np.where(difference > 0, 1, 0)
        psy = np.zeros((len(priceData),))
        psy[:parameter] *= np.nan
        for i in range(parameter, len(priceData)):
            psy[i] = (difference_dir[i-parameter+1:i+1].sum()) / parameter
        psy=psy*100
        inde=list(stock.index.values)
        psy=pd.Series(psy.tolist(),index=inde)
        #print(psy)
        stock[f'PSY{parameter}']= psy
    return stock

In [60]:
PVH = PSY(PVH,parameters)
NKE = PSY(NKE,parameters) 
GPC = PSY(GPC,parameters)
TSCO = PSY(TSCO,parameters)
WMT = PSY(WMT,parameters)
SHW = PSY(SHW,parameters)
HLT = PSY(HLT,parameters)
MAR = PSY(MAR,parameters)
CCL = PSY(CCL,parameters)
NCLH = PSY(NCLH,parameters)
RCL = PSY(RCL,parameters)
V = PSY(V,parameters)
MA = PSY(MA,parameters)
CPRI = PSY(CPRI,parameters)
PVH_new = PSY(PVH_new,parameters)

<h2>SAR:Parabolic SAR</h2>

. SAR trails price as the trend extends over time.


In [61]:
parameters=[(0.02,0.2),(0.02,0.1),(0.01,0.2)]

In [62]:
def SAR(stock,parameters):
    for parameter in parameters:
        sar = talib.SAR(stock['High'], stock['Low'], acceleration=parameter[0], maximum=parameter[1])
        stock[f'SAR{parameter}'] = sar
    return stock

In [63]:
PVH = SAR(PVH,parameters)
NKE = SAR(NKE,parameters) 
GPC = SAR(GPC,parameters)
TSCO = SAR(TSCO,parameters)
WMT = SAR(WMT,parameters)
SHW = SAR(SHW,parameters)
HLT = SAR(HLT,parameters)
MAR = SAR(MAR,parameters)
CCL = SAR(CCL,parameters)
NCLH = SAR(NCLH,parameters)
RCL = SAR(RCL,parameters)
V = SAR(V,parameters)
MA = SAR(MA,parameters)
CPRI = SAR(CPRI,parameters)
PVH_new = SAR(PVH_new,parameters)

In [64]:
PVH.to_csv("PVH+TI.csv")
NKE.to_csv("NKE+TI.csv") 
GPC.to_csv("GPC+TI.csv")
TSCO.to_csv("TSCO+TI.csv")
WMT.to_csv("WMT+TI.csv")
SHW.to_csv("SHW+TI.csv")
HLT.to_csv("HLT+TI.csv")
MAR.to_csv("MAR+TI.csv")
CCL.to_csv("CCL+TI.csv")
NCLH.to_csv("NCLH+TI.csv")
RCL.to_csv("RCL+TI.csv")
V.to_csv("V+TI.csv")
MA.to_csv("MA+TI.csv")
CPRI.to_csv("CPRI+TI.csv")
PVH_new.to_csv('PVHnew+TI.csv')